In [3]:
# imports
import torch
import torchvision
from torch.utils.tensorboard import SummaryWriter
from torchvision import datasets, transforms
%load_ext tensorboard 

C:\Users\USER\anaconda3\envs\chesnn\lib\site-packages\torchvision\io\image.py:11: UserWarning: Failed to load image Python extension: Could not find module 'C:\Users\USER\anaconda3\envs\chesnn\Lib\site-packages\torchvision\image.pyd' (or one of its dependencies). Try using the full path with constructor syntax.
  warn(f"Failed to load image Python extension: {e}")


In [4]:
#!taskkill /IM "tensorboard.exe" /F
!rmdir /S /Q %temp%\.tensorboard-info
%tensorboard --logdir=runs --host localhost

In [5]:
# data params
data_params = {'batch_size':128, 
               'shuffle':True}

# validation params
val_params = {'batch_size': 1, 'shuffle': False}

In [6]:
import os 

# dataset 
class Dataset(torch.utils.data.Dataset):
  def __init__(self, samples, list_IDs, labels):
        'Initialization'
        self.samples = samples
        self.labels = labels
        self.list_IDs = list_IDs

  def __len__(self):
        'Denotes the total number of samples'
        return len(self.list_IDs)

  def __getitem__(self, index):
        'Generates one sample of data'
        # Select sample
        ID = self.list_IDs[index]

         # Load data and get label
        X = self.samples[ID]
        y = self.labels[ID]
        return X, y
    
samples = torch.load('../data/X.pt')
partition = {'train': list(range(17000)), 'dev': list(range(17000, 18000)), 'test': []}
labels = torch.load('../data/Y.pt')
# data loader
training_set = Dataset(samples, partition['train'], labels)
training_generator = torch.utils.data.DataLoader(training_set, **data_params)

validation_set = Dataset(samples, partition['dev'], labels)
validation_generator = torch.utils.data.DataLoader(validation_set, **val_params)

In [27]:
# set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [30]:
# nn model
class EloNN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.layers = torch.nn.Sequential(
            torch.nn.Linear(50, 100),
            torch.nn.ReLU(),
            torch.nn.Linear(100, 25),
            torch.nn.ReLU(),
            torch.nn.Linear(25, 2),
        )
    
    def forward(self, ml_in):
        return self.layers(ml_in)
    
model = EloNN()

In [9]:
# hyper params
learning_rate=1e-3
epochs=150 # 2000
l2=1e-5

In [10]:
# helper function
def get_num_correct(predicted, actual):
    count = 0
    for i, p in enumerate(predicted):
        if abs(p[0] - actual[i][0]) < 100 and abs(p[1] - actual[i][1]) < 100:
            count += 1
    return count

In [31]:
model.load_state_dict(torch.load('multi_layer_perceptron.pt'))
model.train()
model.to(device)
# train model
tb = SummaryWriter()

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=l2)
criterion = torch.nn.MSELoss(reduction='mean')

tb = SummaryWriter()
has_graphed = False

for epoch in range(epochs):

    total_loss = 0
    total_correct = 0

    for inputs, labels in training_generator:
        inputs = inputs.float()
        labels = labels.float()
        inputs, labels = inputs.to(device), labels.to(device)
        preds = model(inputs)
        if epoch == 0 and not has_graphed:
            tb.add_graph(model, inputs)
            has_graphed = True
        
        loss = criterion(preds, labels)
        total_loss += loss.item()
        total_correct += get_num_correct(preds, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    tb.add_scalar("Loss", total_loss, epoch)
    tb.add_scalar("Correct", total_correct, epoch)
    tb.add_scalar("Accuracy", total_correct / len(training_generator), epoch)

    tb.add_histogram("linear1.bias", model.layers[0].bias, epoch)
    tb.add_histogram("linear1.weight", model.layers[0].weight, epoch)
    tb.add_histogram("linear2.bias", model.layers[2].bias, epoch)
    tb.add_histogram("linear2.weight", model.layers[2].weight, epoch)
    tb.add_histogram("linear3.bias", model.layers[4].bias, epoch)
    tb.add_histogram("linear3.weight", model.layers[4].weight, epoch)
        
    tb.add_hparams(
        {"lr": learning_rate, "bsize": data_params['batch_size'], "shuffle": data_params['shuffle']},
        {
            "accuracy": total_correct / len(training_generator),
            "loss": total_loss,
        },
    )

    print("epoch:", epoch, "total_correct:", total_correct, "loss:", total_loss)
    
tb.close()
torch.save(model.state_dict(), 'multi_layer_perceptron.pt')

epoch: 0 total_correct: 1892 loss: 9478631.10546875
epoch: 1 total_correct: 1957 loss: 8611292.97265625
epoch: 2 total_correct: 1971 loss: 8726735.96875
epoch: 3 total_correct: 1926 loss: 8921355.72265625
epoch: 4 total_correct: 1953 loss: 9031657.0546875
epoch: 5 total_correct: 1867 loss: 9809489.234375
epoch: 6 total_correct: 1857 loss: 9172222.84765625
epoch: 7 total_correct: 1915 loss: 9401180.875
epoch: 8 total_correct: 2021 loss: 8610289.2578125
epoch: 9 total_correct: 1922 loss: 9032442.8203125
epoch: 10 total_correct: 1974 loss: 8805939.60546875
epoch: 11 total_correct: 1911 loss: 8688881.7578125
epoch: 12 total_correct: 1924 loss: 9404369.34765625
epoch: 13 total_correct: 1951 loss: 8819103.453125
epoch: 14 total_correct: 2004 loss: 8791510.6171875
epoch: 15 total_correct: 1983 loss: 8823089.3046875
epoch: 16 total_correct: 1988 loss: 8860884.484375
epoch: 17 total_correct: 1958 loss: 8705857.671875
epoch: 18 total_correct: 1921 loss: 9122762.91015625
epoch: 19 total_correct: 

In [32]:
import numpy as np

# validate model
device = 'cpu'
model.to(device)
model.eval()
dev_num = len(validation_generator)
predictions = np.empty((dev_num, 2))
divs_white = np.empty(dev_num)
divs_black = np.empty(dev_num)
i = 0
for val_input, val_label in validation_generator:
    val_input = val_input.float()
    val_input = val_input.to(device)
    y_hat = model(val_input)
    y_hat = y_hat.detach().numpy()[0]
    val_label = val_label.float().detach().numpy()[0]
    print(y_hat)
    print(val_label)
    div_white = y_hat[0] - val_label[0]
    div_black = y_hat[1] - val_label[1]
    predictions[i] = y_hat
    divs_white[i] = div_white
    divs_black[i] = div_black
    print(
        f'prediction = {y_hat}, actual = {val_input}, '
        f'div white = {div_white}, div black = {div_black}'
    )
    i = i + 1
str_res = f'\nstatistics: \nstd white = {np.std(divs_white)}, std black = {np.std(divs_black)} \n' \
      f'mean white = {np.mean(divs_white)}, mean black = {np.mean(divs_black)} \n' \
      f'max white = {np.max(divs_white)}, max black = {np.max(divs_black)} \n' \
      f'min white = {np.min(divs_white)}, min black = {np.min(divs_black)}'
print(str_res)

[2334.1104 2316.5513]
[2619. 2370.]
prediction = [2334.1104 2316.5513], actual = tensor([[ 26.,  60.,  69.,  50.,  51.,  68.,  49.,  39.,  28.,  68.,  65.,  84.,
          73.,  80.,  62.,  98.,  76., 103., 108.,  97.,  39.,  50.,  36.,  36.,
          29.,  23.,   0.,  40.,  44.,  44.,  44.,  70.,  74.,  67.,  79.,  94.,
          64.,  94.,  35.,  35.,  55.,  34.,  38.,  29.,  32.,  37.,   6.,  27.,
          21.,   0.]]), div white = -284.8896484375, div black = -53.44873046875
[2255.2317 2382.0183]
[2191. 2300.]
prediction = [2255.2317 2382.0183], actual = tensor([[  18.,   13.,    9.,   41.,   45.,   46.,   33.,   24.,   13.,   22.,
           20.,   29.,   -1.,   25.,  -19.,   -1.,    0.,   -4.,   22.,   13.,
            3.,   47.,   38.,   30.,    0.,    0.,    0.,    0.,  -22.,    6.,
          -26.,  -44.,  -45.,  -51.,  -39.,  -47.,  -44.,  -31.,  -94., -108.,
         -112., -114., -132., -101., -113., -102.,  -96., -123., -116., -123.]]), div white = 64.231689453125, div bl